In [4]:
# import os
# import shutil
# import random

# # Paths to your dataset directories
# training_data_directory = 'D:/GMU/FALL2024/AIT 736/Final_project/Dataset/Training'
# validation_data_directory = 'D:/GMU/FALL2024/AIT 736/Final_project/Dataset/Validation'

# # Create validation folders if they don't exist
# os.makedirs(validation_data_directory, exist_ok=True)
# for class_name in os.listdir(training_data_directory):
#     os.makedirs(os.path.join(validation_data_directory, class_name), exist_ok=True)

# # Percentage of data to use for validation
# validation_split = 0.2

# # Loop through each class folder in the training set
# for class_name in os.listdir(training_data_directory):
#     class_path = os.path.join(training_data_directory, class_name)
#     images = os.listdir(class_path)
    
#     # Shuffle and select a subset of images for validation
#     random.shuffle(images)
#     validation_size = int(len(images) * validation_split)
#     validation_images = images[:validation_size]
    
#     # Move selected images to validation folder
#     for img in validation_images:
#         source = os.path.join(class_path, img)
#         destination = os.path.join(validation_data_directory, class_name, img)
#         shutil.move(source, destination)

# print("Validation set created successfully!")


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore 


# Create an instance of ImageDataGenerator for training with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,             # Normalize pixel values to range [0, 1]
    rotation_range=20,          # Randomly rotate images within 20 degrees
    width_shift_range=0.2,      # Randomly shift the image width
    height_shift_range=0.2,     # Randomly shift the image height
    shear_range=0.2,            # Shear transformation (distortion)
    zoom_range=0.2,             # Random zoom-in and zoom-out
    horizontal_flip=True        # Flip the images horizontally
)

# Create training data generator
train_generator = train_datagen.flow_from_directory(
    'D:/GMU/FALL2024/AIT 736/Final_project/Dataset/Training',  # Replace with your actual training data path
    target_size=(224, 224),             # Resize images to 224x224 (VGG16 input size)
    batch_size=32,                      # Number of images per batch
    class_mode='categorical',           # Use 'categorical' for multi-class classification
    subset='training'
)


Found 4571 images belonging to 4 classes.


In [6]:
# Create a separate ImageDataGenerator for validation without augmentation
validation_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation data

# Create validation data generator
validation_generator = validation_datagen.flow_from_directory(
    'D:/GMU/FALL2024/AIT 736/Final_project/Dataset/Validation',  # Path to your separate validation data folder
    target_size=(224, 224),     # Resize images to 224x224 (VGG16 input size)
    batch_size=32,
    class_mode='categorical'    # Use 'categorical' for multi-class classification
)


Found 1141 images belonging to 4 classes.


In [7]:
# Create a separate ImageDataGenerator for testing without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    'D:/GMU/FALL2024/AIT 736/Final_project/Dataset/Testing',     # Path to your separate testing data folder
    target_size=(224, 224),     # Resize images to 224x224 (VGG16 input size)
    batch_size=32,
    class_mode='categorical'    # Use 'categorical' for multi-class classification
)


Found 1311 images belonging to 4 classes.


In [8]:
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.layers import Dense, Flatten, Dropout # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore

# 1. Load the VGG16 model without the top layers
# Weights are pre-trained on ImageNet, and we exclude the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 2. Add custom layers on top of the VGG16 base model
x = base_model.output  # Get the output of the VGG16 base model
x = Flatten()(x)       # Flatten the output to feed it into fully connected layers
x = Dense(256, activation='relu')(x)  # Add a fully connected layer with ReLU activation
x = Dropout(0.5)(x)    # Add a Dropout layer to reduce overfitting

# Add the final output layer with 4 neurons (number of classes) and softmax activation
predictions = Dense(4, activation='softmax')(x)  # Update '4' with your number of classes

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# 3. Freeze the layers of the base VGG16 model to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# 4. Compile the model with an optimizer, loss function, and evaluation metric
model.compile(optimizer=Adam(learning_rate=0.0001),  # Set a small learning rate for fine-tuning
              loss='categorical_crossentropy',       # Suitable loss function for multi-class classification
              metrics=['accuracy'])                 # Use accuracy as the evaluation metric

# Display the model summary to check the layers
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,138,500 (80.64 MB)

 Trainable params: 6,423,812 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [9]:
# Set the number of epochs
epochs = 20

# Train the model using the training and validation generators
history = model.fit(
    train_generator,                # Training data generator
    epochs=epochs,                  # Number of epochs to train
    validation_data=validation_generator,  # Validation data generator
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


c:\Users\sreed\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5663 - loss: 1.0317

c:\Users\sreed\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


142/142 ━━━━━━━━━━━━━━━━━━━━ 342s 2s/step - accuracy: 0.5670 - loss: 1.0303 - val_accuracy: 0.7705 - val_loss: 0.6168
Epoch 2/20
  1/142 ━━━━━━━━━━━━━━━━━━━━ 3:35 2s/step - accuracy: 0.6562 - loss: 0.9073

c:\Users\sreed\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6562 - loss: 0.9073 - val_accuracy: 0.8571 - val_loss: 0.3798
Epoch 3/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 286s 2s/step - accuracy: 0.7784 - loss: 0.5792 - val_accuracy: 0.8580 - val_loss: 0.4227
Epoch 4/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6562 - loss: 0.7315 - val_accuracy: 0.7619 - val_loss: 0.4454
Epoch 5/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 287s 2s/step - accuracy: 0.7991 - loss: 0.5141 - val_accuracy: 0.8482 - val_loss: 0.3880
Epoch 6/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9688 - loss: 0.2515 - val_accuracy: 1.0000 - val_loss: 0.1454
Epoch 7/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 295s 2s/step - accuracy: 0.8188 - loss: 0.4673 - val_accuracy: 0.8804 - val_loss: 0.3474
Epoch 8/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9688 - loss: 0.2275 - val_accuracy: 0.7619 - val_loss: 0.4559
Epoch 9/20
142/142 ━━━━━━━━━━━━━━━━━━━━ 289s 2s/step - accuracy: 0.8323 - loss: 0.4474 - val_accuracy: 0.8741 - 

In [10]:
# Step 3: Retrain the Model with Fine-Tuning

# Unfreeze specific layers for fine-tuning
for layer in base_model.layers:
    if layer.name.startswith('block5'):  # Unfreeze layers in block5
        layer.trainable = True
    else:
        layer.trainable = False


In [11]:
# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [12]:
# Fine-tune the model for a few epochs
fine_tune_epochs = 10
total_epochs = 20 + fine_tune_epochs  # Add fine-tuning epochs to previous training epochs

history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=20,  # Start from where the previous training left off
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)


Epoch 21/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.8680 - loss: 0.3537 - val_accuracy: 0.8973 - val_loss: 0.2919
Epoch 22/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8125 - loss: 0.3486 - val_accuracy: 0.9048 - val_loss: 0.1806
Epoch 23/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - accuracy: 0.8779 - loss: 0.3388 - val_accuracy: 0.9098 - val_loss: 0.2443
Epoch 24/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9688 - loss: 0.2612 - val_accuracy: 0.9524 - val_loss: 0.2592
Epoch 25/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - accuracy: 0.8969 - loss: 0.2588 - val_accuracy: 0.9170 - val_loss: 0.2337
Epoch 26/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9062 - loss: 0.3550 - val_accuracy: 0.8095 - val_loss: 0.5164
Epoch 27/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.9098 - loss: 0.2576 - val_accuracy: 0.9205 - val_loss: 0.2205
Epoch 28/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9688 - loss: 0.0956 - val_

In [15]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)

# Print out the results
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Loss: {test_loss}")


41/41 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.9015 - loss: 0.2442
Test Accuracy: 0.9084668159484863
Test Loss: 0.2370285838842392


In [17]:
# Save the model in the recommended native Keras format
model.save('brain_tumor_detection_model.keras')
